In [ ]:
# COLMAP
%cd /notebooks
!pip install -q plyfile pycolmap Pillow==9.5.0 pycolmap plotly

!sudo apt-get update
!sudo apt-get install libglm-dev

!pip install -q /notebooks/diff-gaussian-rasterization
!pip install -q /notebooks/gaussian-splatting/submodules/simple-knn

!apt install imagemagick -y
# remove line 88 of vim ~/../etc/ImageMagick-6/policy.xml to run MoviePy
!sed -i '88d' ~/../etc/ImageMagick-6/policy.xml 

!sudo apt-get update && sudo apt-get install -y \
	ninja-build \
	libeigen3-dev \
	libflann-dev \
	libsqlite3-dev \
	libgl1-mesa-dev \
	cmake \
	libgoogle-glog-dev \
	libgflags-dev \
	libsuitesparse-dev \
	libceres-dev \
	libcgal-dev \
	libcgal-qt5-dev \
	libmetis-dev \
	libfreeimage-dev \
	libglew-dev

!sudo updatedb   # update the database used by locate
!locate Eigen3Config.cmake
!locate libflann

# # # Build COLMAP
# !git clone https://github.com/colmap/colmap.git
# %cd /notebooks/colmap
# !rm -rf build
# !mkdir build
# %cd /notebooks/colmap/build
# !cmake .. -DCMAKE_CUDA_ARCHITECTURES=native -GNinja

%cd /notebooks/colmap/build
!ninja
!sudo ninja install

# Check if colmap is installed
!colmap -h

%cd /notebooks


#HLOC
!pip install mediapy

# !rm -rf /notebooks/Hierarchical-Localization
# !git clone --quiet --recursive https://github.com/cvg/Hierarchical-Localization/

%cd /notebooks/Hierarchical-Localization
!pip install --progress-bar off --quiet -e .


%cd /notebooks

In [ ]:
# Change to the appropriate directory
%cd /notebooks/Hierarchical-Localization

# Import libraries from hloc
from hloc import (
	extract_features, 
	match_features, 
	reconstruction, 
	visualization, 
	pairs_from_retrieval, 
	colmap_from_nvm, 
	pairs_from_exhaustive, 
	pairs_from_covisibility
)
from hloc.visualization import plot_images, read_image
from hloc.utils import viz_3d

# Update system path
import sys
third_party_path = '/notebooks/Hierarchical-Localization/third_party'
sys.path.extend([
	third_party_path, 
	f'{third_party_path}/d2net',
	f'{third_party_path}/deep-image-retrieval',
	f'{third_party_path}/r2d2',
	f'{third_party_path}/SuperGluePretrainedNetwork'
])

# Navigate back to the notebooks directory
%cd /notebooks

# Import standard libraries
import os
import io
import glob
import shutil
import cv2
import sqlite3
import numpy as np
import torch
import matplotlib.pyplot as plt
from pathlib import Path
from datetime import datetime, timedelta
from PIL import Image, ImageFilter, ImageChops
import mediapy as media
import pycolmap

In [ ]:
# Connect to the database
%cd /notebooks
connection = sqlite3.connect('/notebooks/dbs/db_session_EE3B13AA-73DD-415A-B0D6-908745325E92.sqlite')

# # Define the search parameters
# tl_lat = 160.7128
# tl_long = -74.0060
# br_lat = 10.6977
# br_long = -13.9792

# cursor = connection.cursor()
# cursor.execute("SELECT image_blob FROM maptodon_raw WHERE lat BETWEEN ? AND ? AND long BETWEEN ? AND ?", (br_lat, tl_lat, tl_long, br_long))
# results = cursor.fetchall()

# Get all data
cursor = connection.cursor()
cursor.execute("SELECT image_blob FROM maptodon_raw")
results = cursor.fetchall()

# Delete working image, and create a new one
!rm -rf working
!mkdir working
!mkdir working/tmp_input

# Process the search results
image_hashes = {}
if len(results) > 0:
	for i, row in enumerate(results):
		image_data = row[0]

		# Save to working directory
		image = Image.open(io.BytesIO(image_data))

		image.save("./working/tmp_input/image_{:05d}.png".format(i))
	print("Saved " + str(len(results)) + " images to working directory.")
else:
	print("No results found.")

# Close the database connection
connection.close()

In [ ]:
!rm -rf ./working/input
!mkdir ./working/input

images = glob.glob("./working/tmp_input/*.png")

# Maximum number of images to copy
M = 100
step = len(images) // M

for i, image in enumerate(images):
	if i % step == 0:
		shutil.copy(image, f"./working/input/{i:05d}.png")
images = glob.glob("./working/input/*.png")
print(f"There are {len(images)} images in the working directory.")

In [ ]:
%%time
%cd /notebooks

location = Path('./working')
images = location / 'input'
image_list = [path.split(str(images.stem) + "/")[-1] for path in sorted(glob.glob(str(images / '*')))]
outputs = location / 'sfm'
sfm_pairs = outputs / 'pairs-sfm.txt'
loc_pairs = outputs / 'pairs-loc.txt'
sfm_dir = outputs / 'sfm'
features = outputs / 'features.h5'
matches = outputs / 'matches.h5'

feature_conf = extract_features.confs['superpoint_max']
matcher_conf = match_features.confs['superglue']

extract_features.main(feature_conf, images, image_list=image_list, feature_path=features)
pairs_from_exhaustive.main(sfm_pairs, image_list=image_list)
match_features.main(matcher_conf, sfm_pairs, features=features, matches=matches)

model = reconstruction.main(sfm_dir, images, sfm_pairs, features, matches, image_list=image_list)

fig = viz_3d.init_figure()
viz_3d.plot_reconstruction(fig, model, color='rgba(255,0,0,0.5)', name="mapping", points_rgb=True)
fig.show()

In [ ]:
%%time
print("\nRunning COLMAP image undistorter...\n")

# Running COLMAP image undistorter
undistorted_path = location / "undistorted"
!rm -rf {undistorted_path}

!colmap image_undistorter \
	--image_path /notebooks/working/input \
	--input_path {sfm_dir} \
	--output_path {undistorted_path} \
	--output_type COLMAP \
	# --max_image_size 2000

!mv {undistorted_path}/sparse {undistorted_path}/sparse_tmp
!mkdir {undistorted_path}/sparse
!mv {undistorted_path}/sparse_tmp {undistorted_path}/sparse/0

In [ ]:
!python /notebooks/gaussian-splatting/train.py -s {undistorted_path} -r 2